## Part 2: Building the Fact and Dimension Tables

This notebook takes the cleaned DataFrames from Part 1 and uses them to construct a star schema. The schema will consist of a central `fact_trips` table and four dimension tables: `dim_date`, `dim_transport`, `dim_customer`, and `dim_celestial_object`.

In [5]:
import pandas as pd
import numpy as np
import pickle

with open('cleaned_data.pkl', 'rb') as f:
    dfs_cleaned = pickle.load(f)

# verify that the keys (table names) are correct
print("Available tables:", list(dfs_cleaned.keys()))

Available tables: ['Transport', 'Trip', 'CelestialObject', 'Visit', 'Customer', 'Participant']


In [6]:
# lets create a new dictionary to hold our final schema tables
schema_tables = {}

### Create `dim_date`

This dimension gives context to when a trip occurred. We will create it from the `departure_date` in our cleaned `Trip` table. We'll extract features like year, month, and day of the week, and create a unique integer key for each date.


In [7]:
trip_df = dfs_cleaned['Trip']

# Create the base for the dimension table, dropping duplicates
dim_date = trip_df[['departure_date']].drop_duplicates().reset_index(drop=True)

# feature engineering for the ml model
dim_date['full_date'] = dim_date['departure_date']
dim_date['year'] = dim_date['departure_date'].dt.year
dim_date['month'] = dim_date['departure_date'].dt.month
dim_date['day_of_week'] = dim_date['departure_date'].dt.day_name()

# create the date_key (e.g., 2032-08-17 -> 20320817)
dim_date['date_key'] = dim_date['departure_date'].dt.strftime('%Y%m%d').astype(int)

dim_date = dim_date[['date_key', 'full_date', 'year', 'month', 'day_of_week']]

schema_tables['dim_date'] = dim_date
print("`dim_date` created successfully.")
dim_date.head()

`dim_date` created successfully.


,date_key,full_date,year,month,day_of_week
0,20320817,2032-08-17,2032,8,Tuesday
1,20320813,2032-08-13,2032,8,Friday
2,20300917,2030-09-17,2030,9,Tuesday
3,20321129,2032-11-29,2032,11,Monday
4,20340112,2034-01-12,2034,1,Thursday


### Creating Simple Dimensions

These three dimensions are straightforward. We select the relevant table from our cleaned data and rename the columns to match our star schema design (using `_key` and `_description`/`_name`).

In [8]:
# Dimension: Transport
dim_transport = dfs_cleaned['Transport'].rename(columns={
    'code': 'transport_key',
    'description': 'transport_description'
})
schema_tables['dim_transport'] = dim_transport


# Dimension: Customer
dim_customer = dfs_cleaned['Customer'].rename(columns={
    'number': 'customer_key',
    'name': 'customer_name'
})
schema_tables['dim_customer'] = dim_customer


# Dimension: Celestial Object
dim_celestial_object = dfs_cleaned['CelestialObject'].rename(columns={
    'name': 'celestial_object_key'
})
schema_tables['dim_celestial_object'] = dim_celestial_object

schema_tables['dim_transport']

,transport_key,transport_description
0,RV,space shuttle
1,BU,beam up
2,FTL,faster than light cruiser


In [9]:
schema_tables['dim_customer'].head()

,customer_key,customer_name,birth_date
0,1,Janice Smith,1990-08-12
1,2,Michael Stout,2003-11-08
2,3,Amy Davis,2006-06-18
3,4,Stephanie Lester,2007-01-12
4,5,Joseph Johnson,2002-09-19


In [10]:
schema_tables['dim_celestial_object'].head()

,celestial_object_key,parent_object,distance_from_parent_1000km,diameter_km
0,Sun,None,0.0,1393000
1,Mercury,Sun,57900.0,4878
2,Venus,Sun,108200.0,12104
3,Earth,Sun,149600.0,12756
4,Moon,Earth,384.4,3476


### Creating the Fact Table ( Base and Keys)

Now we'll build the central `fact_trips` table. First, we select the base measures (`price`, `duration`) from the cleaned `Trip` table. Then we create the foreign keys that will link to our dimension tables.


In [11]:
fact_trips = dfs_cleaned['Trip'].copy()

# Rename columns to match the schema design
fact_trips = fact_trips.rename(columns={'number': 'trip_key', 'transport': 'transport_key'})

# Create the date_key for joining with dim_date
fact_trips['date_key'] = fact_trips['departure_date'].dt.strftime('%Y%m%d').astype(int)

# We only need the keys and measures in the fact table
fact_trips = fact_trips[['trip_key', 'date_key', 'transport_key', 'price', 'duration']]

fact_trips.head()

,trip_key,date_key,transport_key,price,duration
0,1,20320817,BU,17186.78,368.0
1,2,20320813,FTL,18551.89,237.0
2,3,20300917,RV,19391.71,314.0
3,4,20321129,FTL,27844.02,299.0
4,5,20340112,FTL,5393.89,113.0


### Creating the Fact Table (Calculated Facts)

Our schema design includes calculated facts: `total_passengers` and `total_stay_duration`. We need to calculate these by aggregating data from the `Participant` and `Visit` tables, respectively.


In [12]:
# calculate total passengers per trip
passengers_per_trip = dfs_cleaned['Participant'].groupby('trip')['customer'].count().reset_index()
passengers_per_trip = passengers_per_trip.rename(columns={'trip': 'trip_key', 'customer': 'total_passengers'})

# calculate total stay duration per trip
stay_per_trip = dfs_cleaned['Visit'].groupby('trip')['stay_duration'].sum().reset_index()
stay_per_trip = stay_per_trip.rename(columns={'trip': 'trip_key', 'stay_duration': 'total_stay_duration'})


print("\n--- Passengers per Trip ---")
print(passengers_per_trip.head())

print("\n--- Stay Duration per Trip ---")
print(stay_per_trip.head())


--- Passengers per Trip ---
   trip_key  total_passengers
0         1                 1
1         2                11
2         3                 6
3         4                12
4         5                12

--- Stay Duration per Trip ---
   trip_key  total_stay_duration
0         1                  366
1         2                  234
2         3                  309
3         4                  295
4         5                  111


### Create the Fact Table (Part 3 - Final Merge)

Finally, we merge our calculated facts into the main `fact_trips` table using the `trip_key`.

In [13]:
# Merge the calculated facts into the fact table
fact_trips = pd.merge(fact_trips, passengers_per_trip, on='trip_key', how='left')
fact_trips = pd.merge(fact_trips, stay_per_trip, on='trip_key', how='left')

# Handle cases where a trip might have no recorded visits or passengers
fact_trips['total_passengers'].fillna(0, inplace=True)
fact_trips['total_stay_duration'].fillna(0, inplace=True)

# Convert to integer types
fact_trips['total_passengers'] = fact_trips['total_passengers'].astype(int)
fact_trips['total_stay_duration'] = fact_trips['total_stay_duration'].astype(int)

schema_tables['fact_trips'] = fact_trips
print("Final `fact_trips` table created successfully.")

Final `fact_trips` table created successfully.


C:\Users\erwin\AppData\Local\Temp\ipykernel_22008\3268452021.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fact_trips['total_passengers'].fillna(0, inplace=True)
C:\Users\erwin\AppData\Local\Temp\ipykernel_22008\3268452021.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

### Final Verification

Let's look at the head of our completed `fact_trips` table. It should contain the primary key, all foreign keys, and all the measure columns. The data is now structured in a clean star schema.


In [14]:
schema_tables['fact_trips'].head()

,trip_key,date_key,transport_key,price,duration,total_passengers,total_stay_duration
0,1,20320817,BU,17186.78,368.0,1,366
1,2,20320813,FTL,18551.89,237.0,11,234
2,3,20300917,RV,19391.71,314.0,6,309
3,4,20321129,FTL,27844.02,299.0,12,295
4,5,20340112,FTL,5393.89,113.0,12,111


In [15]:
import pickle

# The 'wb' means 'write bytes'.
with open('schema_tables.pkl', 'wb') as f:
    pickle.dump(schema_tables, f)

print("Schema tables dictionary has been saved to 'schema_tables.pkl'")

Schema tables dictionary has been saved to 'schema_tables.pkl'
